# Valuación de opciones con Black-Scholes

Este trabajo utiliza fórmulas para determinar:

* el precio de una opción dada su volatilidad.
* la volatilidad implícita en el precio de una opción. 

In [16]:
import numpy as np
import pandas as pd
import datetime as dt
import scipy.stats as scs

Estos son los paquetes a utilizar.  
En caso de no poseer alguno de ellos, estos pueden ser instalados por medio del comando  
! pip install {nombre_del_paquete}

In [8]:
def BSM_call_price(p_last:float, strike:float, TTM:float, rf:float, vol:float):
    d1 = (np.log(p_last / strike) + (rf + 1/2 * vol ** 2) * TTM) / (vol * np.sqrt(TTM))
    d2 = (np.log(p_last / strike) + (rf - 1/2 * vol ** 2) * TTM) / (vol * np.sqrt(TTM))
            
    call_price = (p_last * scs.norm.cdf(d1, 0, 1) - strike * np.exp(-rf * TTM) * scs.norm.cdf(d2, 0, 1))
    
    return call_price, d1, d2

Esta es la función que toma como inputs las variables de la acción y calcula su valor teórico según Black Scholes.  

Veamos un ejemplo, una opción cuyo subyacente cotiza a `$`100, un precio de ejercicio de `$`110, 6 meses a su expiración, una tasa de interés del 20`%` y una volatilidad del 25`%` anual.

In [11]:
call_price, _, _ = BSM_call_price(100, 110, 0.5, 0.2, 0.25)
print(call_price)

7.263128976480701


In [9]:
def BSM_imp_vol(call_price:float, p_last:float, strike:float, TTM:float, rf:float):
    iv = 0.1
    iters = 1
    
    while iters < 1000:
        result, d1, d2 = BSM_call_price(p_last, strike, TTM, rf, iv)
        
        if result > call_price:
            iv = iv-0.005
            iters = iters+1
        
        elif result < call_price:
            iv = iv+0.005
            iters = iters+1
        
        else:
            break
            
    if (call_price - result > call_price*0.05):
        raise Warning
    
    return iv

Esta función es la que calcula la volatilidad implícita de una opción, tomando el precio como dado.  

Usaremos esta función para reversar la operación anterior.  
Es decir, estimaremos la volatilidad implícita de una opción con las mismas características, y con el precio que recibimos del último calculo.

In [14]:
iv = BSM_imp_vol(call_price, 100, 110, 0.5, 0.2)
print(iv)

0.2500000000000001


Vemos así que esta es una operación reversible.  
Utilizaremos estas mismas fórmulas próximamente para una implementación en objetos de opciones de compra.